In [ ]:
from twikit import Client, TooManyRequests
from account_info import USERNAME, EMAIL, PASSWORD
import pandas as pd
import json
import urllib.request
import os
import time
import requests
import re

# client = Client('en-US')
# client.login( 
#     auth_info_1='god_comput74080' ,
#     auth_info_2='computerscienceenrichment@outlook.com',
#     password=PASSWORD
# )

client = Client('en-US')

client.login( 
    auth_info_1='TonyJ21128' ,
    auth_info_2='notredamelogin@outlook.com',
    password=PASSWORD
)

client2 = Client('en-US')

client2.login( 
    auth_info_1='turo343994' ,
    auth_info_2='summerenrichmentprogramcs@outlook.com',
    password='Molina.2005'
)

# client3 = Client('en-US')

# client3.login( 
#     auth_info_1='tony00551251172' ,
#     auth_info_2='tonystarkwon@outlook.com',
#     password=PASSWORD
# )

# save cookies in order to pull data without getting banned
# client.save_cookies('cookies.json')
client.save_cookies('cookies1.json')
client2.save_cookies('cookies2.json')
# client3.save_cookies('cookies3.json')

In [ ]:
# client5 = Client('en-US')

# client5.login( 
#     auth_info_1='LebronJack27791',
#     auth_info_2='project2forser@outlook.com',
#     password=PASSWORD
# )
client4 = Client('en-US')

client4.login( 
    auth_info_1='StarkMiche4868' ,
    auth_info_2='project1forser@outlook.com',
    password=PASSWORD
)


client6 = Client('en-US')

client6.login( 
    auth_info_1='sanches50767' ,
    auth_info_2='project3forser@outlook.com',
    password=PASSWORD
)


client7 = Client('en-US')

client7.login( 
    auth_info_1='tonymolnar83096' ,
    auth_info_2='project4forser@outlook.com',
    password=PASSWORD
)

client8 = Client('en-US')
client8.login( 
    auth_info_1='michealjon77794' ,
    auth_info_2='project5forser@outlook.com',
    password=PASSWORD
)
client4.save_cookies('cookies4.json')
client5.save_cookies('cookies5.json')
client6.save_cookies('cookies6.json')
client7.save_cookies('cookies7.json')
client8.save_cookies('cookies8.json')

In [ ]:
# file path to excel sheet
file_path_to_excel = r"C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\codes.xlsx"

In [ ]:
def download_image(URL, save_as):
    '''Function uses image URL and saves the image onto the desired path and file type'''
    urllib.request.urlretrieve(URL,save_as)


def delete_jpg(file):
    '''This file takes in the file path and deletes it.'''
    if os.path.exists(file):
        os.remove(file)
    else:
        print('File not found')


# this saves the json file on the described directory 
def save_json(file, file_path):
    with open(file_path, "w") as f:
        json.dump(file, f,indent=4)


def open_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data 


def retry_on_rate_limit_error(func, *args, **kwargs):
    max_retries = 5
    retries = 0
    while retries < max_retries:
        try:
            return func(*args, **kwargs)
        except TooManyRequests as e:
            print("Rate limit exceed trying again in 60 sec")
            time.sleep(60)
            retries += 1
    raise Exception("Max tries reached")


In [ ]:


def keywords_to_dict(file_path =r"C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\codes.xlsx" ):
    '''This function takes in the excel sheet that has keywords and its ids split up into columns.
    Then it creates two dictionaries, keyname and keyname_id. The values are numbered 0 to the 
    length of the columns. Once created they return these dictionaries.'''

    # reads in excel sheet into pandas data frame
    df = pd.read_excel(file_path)

    # this turns the street name and its code name into list 
    street_name_list = df['keyname'].astype(str).values.tolist()
    street_name_code_list = df['keyname_id'].astype(str).values.tolist()

    # this dictionary contains the keyname in the keys and keyname id in values
    keyname = {}
    keyname_id = {}
    for i in range(33):
        keyname[i] = street_name_list[i]
        keyname_id[i] = street_name_code_list[i]
        
    return keyname, keyname_id

keyname, keyname_id = keywords_to_dict()
print(keyname)

In [ ]:

def keyword_search_in_users(keyname_to_search, list_of_users, list_to_append, keywords_dict, iteration):
    """This function takes in a keyname to search a list of users. It will search the users name,
    keyname, description and tweets for the keyname. If it finds it on the tweets it appends
    to the list_to_append"""

    keynames, keyname_ids = keywords_to_dict()
    keyword_id = keyname_ids[iteration]
    # gets a user from the list_of_users and goes through their name, screen name 
    # and description to find keywords. If they find one they append the user to the list]
    
    for user in list_of_users:
        if len(list_to_append) == 20:
            break
        else: 
            if keyname_to_search in user.name:
                keywords_dict[keyword_id]["ids"].append('user_'+ str(user.id))
                list_to_append.append('user_' +  user.id)

            elif keyname_to_search in user.screen_name:
                keywords_dict[keyword_id]["ids"].append('user_'+ str(user.id))
                list_to_append.append('user_' + user.id)

            elif keyname_to_search in user.description:
                keywords_dict[keyword_id]["ids"].append('user_'+ str(user.id))
                list_to_append.append('user_' + user.id)

    return list_to_append, keywords_dict


def keyword_search_in_tweets(tweet):
    """This function takes in tweets, keyname, and keyname ids. Then 
    it goes thourgh the keynames and trys to find it in the text of the 
    tweet. If it finds it appends it to a list and at the end it returns this
    list."""

    keyname, keyname_id = keywords_to_dict()
    keyname_found_in_tweet = []

    # find keyword in tweet text, if so append to list
    for keys, keyword in keyname.items():
        if keyword in tweet.full_text:
            keyname_found_in_tweet.append(keyname_id[keys])
    
    return keyname_found_in_tweet


def relations_in_comments(tweet, reply, relations):
            
            Relation_Structure = {
                "src_id": 'tweet_' + str(reply.id),
                "relation": 'comment-under-post',
                "dest_id": 'tweet_' + str(tweet.id)
            }

            reply_user = reply.user
            relations.append(Relation_Structure)

            Relation_Structure = {
                "src_id": 'user_' + str(reply_user.id),
                "relation": 'user-make-comment',
                "dest_id": 'tweet_' + str(reply.id)
            }

            relations.append(Relation_Structure)

            keywords, keywords_ids = keywords_to_dict()

            for key, keyname in keywords.items():
                if keyname in reply.full_text:
                    Relation_Structure = {
                        "src_id": 'tweet_' + str(reply.id),
                        "relation": 'comment-contain-keyword',
                        "dest_id": keywords_ids[key]
                    }

                    relations.append(Relation_Structure)
            return relations


def remove_duplicates_from_list(list):
    """This functions removes duplicates from list"""
    result = []
    for item in list:
        if item not in result:
            result.append(item)
    return result


def tweet_mentions_user(user, tweet, relations):
    """This function is for when a user mentions another user in a tweet. Finds 
    mentioned users user_name and adds the relation_structure to relations."""

    mentions = re.findall(r'@(\S+)', tweet.full_text)
    mentions = remove_duplicates_from_list(mentions)
    for mention in mentions:
        try:
            mentioned_user = client.get_user_by_screen_name(mention)
            Relation_Structure = {
                "src_id": 'user_' + str(user.id),
                "relation": 'user-mention/tag-user',
                "dest_id": 'user_' + str(mentioned_user.id)
            }

            relations.append(Relation_Structure)
        except:
            pass
    return relations 


def extract_tweet_picture_structure(user, tweet, pic_id_starter, relations):
    """This function takes in the type user and tweet. Then goes down the post
    structure sorting out the data. Finally it returns the post_structure"""
    
    if '@' in tweet.full_text:
        relations = tweet_mentions_user(user, tweet, relations)

    # retrieve keywords found in tweet
    keyname_found = keyword_search_in_tweets(tweet)
    # construct post structure
    favoriters = tweet.get_favoriters(20)

    Relation_Structure = {
            "src_id": 'user_' + str(user.id),
            "relation": 'user-publish-post',
            "dest_id": 'tweet_' + str(tweet.id)
        }

    relations.append(Relation_Structure)

    Post_Structure = {
        "user_id": 'user_' + str(user.id),
        "post_id": 'tweet_' + str(tweet.id),
        "user_comment": tweet.full_text,
        "pic_id": "",
        "liked_users": [ 'user_' + str(favoriter.id) for favoriter in favoriters],
        "comments": '',
        "keywords": keyname_found,
    }
    if keyname_found:
        tweet = client.get_tweet_by_id(tweet.id)
        replies = tweet.replies
        if replies:
            Post_Structure["comments"] = [f'tweet_{reply.id}' for reply in replies]
            
            for reply in replies:
                relations = relations_in_comments(tweet, reply, relations)
    elif tweet.replies:
        replies = tweet.replies
        if replies:
            Post_Structure["comments"] = [f'tweet_{reply.id}' for reply in replies]
            
            for reply in replies:
                relations = relations_in_comments(tweet, reply, relations)
            
    for favoriter in favoriters:
        Relation_Structure = {
                "src_id": 'user_' + str(favoriter.id),
                "relation": 'user-like-post',
                "dest_id": 'tweet_' + str(tweet.id)
            }
        
        relations.append(Relation_Structure)
         
    media_data = tweet.media
    if media_data:
        Post_Structure["pic_id"] = 'pic_' + str(pic_id_starter)

        Picture_Structure = {
            "pic_id": 'pic_' + str(pic_id_starter),
            "post_id": 'tweet_' + str(tweet.id),
            "url": media_data[0].get('media_url_https')
        }

        Relation_Structure = {
                "src_id": 'tweet_' + str(tweet.id),
                "relation": 'post-has-picture',
                "dest_id": 'post_' + str(pic_id_starter)
            }
        
        relations.append(Relation_Structure)
        pic_id_starter += 1
        return Post_Structure, Picture_Structure, pic_id_starter, relations
    
    else:
        Picture_Structure = {}
        return Post_Structure, Picture_Structure, pic_id_starter, relations


def get_relations_followers_following(relations, followers_list, user):
    '''This function gets the realtions between a user and it's
    following/follower list'''

    user_id = user.id
    for follower in followers_list:
        Relation_Structure = {
                "src_id": follower,
                "relation": 'user-follow/followed-user',
                "dest_id": 'user_' + str(user_id)
            }
                
        relations.append(Relation_Structure)
    return relations
    

def realtion_in_profile(user, keyword_id, relations):
    '''Finds the relations profile of users. If a keyword is found 
    in the profile it creates a relation structure for it.'''
    Relation_Structure = {
                "src_id": 'user_' + str(user.id),
                "relation": 'user-profile-keyword',
                "dest_id": keyword_id
            }
    relations.append(Relation_Structure)
    return relations


def realtion_keyword_in_tweet(tweet, keyword_id, relations):
    '''Adds relation structure between post and keyword'''
    Relation_Structure = {
                "src_id": 'tweet_' + str(tweet.id),
                "relation": 'post-include-keyword',
                "dest_id": keyword_id,
            }
    relations.append(Relation_Structure)
    return relations


def add_followers_or_following(followers_list, followers):
    if len(followers_list) != 10:
        for follower in followers:
            if len(followers_list) == 10:
                break
            elif follower not in followers_list:
                followers_list.append('user_' + follower.id)
    return followers_list

def user_get_followers(user):
    followers = user.get_followers(10)
    return followers


def user_get_following(user):
    following = user.get_following(10)
    return following

In [ ]:
def try_get_user_following(user):
    try:
        followers = user.get_followers(10)
        following = user.get_following(10)
    except:
        followers = []
        following = []
    if len(followers) == 0:
        try:
            followers = client.get_user_followers(user.id,10)
            following = client.get_user_following(user.id,10)
        except:
            followers = []
            following = []
    if len(followers) == 0:
        try:
            followers = user_get_followers(user)
            following = user_get_following(user)
        except:
            followers = []
            following = []
    return followers, following

def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        contents = file.read()
    return contents 


def write_txt_file(file_path, new_contents):
    with open(file_path, 'w') as file:
        file.write(new_contents)
    return 

In [ ]:
def change_client_for_users(counter):
    if counter > 4:
        counter = 1
    if counter == 1:
        client.load_cookies('cookies7.json')
    elif counter == 2:
        client.load_cookies('cookies8.json')
    elif counter == 3:
        client.load_cookies('cookies7.json')
    elif counter == 4:
        client.load_cookies('cookies8.json')

    return counter 

In [ ]:
def creating_structures_for_twitter(user_keyword, pic_id_num_starter, save_path_list):

    account_num = 1
    time_counter = 1
    counter_client= 1
    for i, user in enumerate(user_keyword):
        pic_id_num_starter = int(read_txt_file(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\picture_id_num.txt'))
        users = open_json(save_path_list[0])
        posts = open_json(save_path_list[1])
        pictures = open_json(save_path_list[2])
        keywords_dict = open_json(save_path_list[3])
        relations = open_json(save_path_list[4])
        counter_client = change_client_for_users(counter_client)

        if f'user_{user.id}' in users:
            account_num += 1
            continue
        print(f'entering {account_num}, {user.screen_name}')
        try: 

            keywords = []

            # keyname contains the keynames and keyname_id contains their ids
            keyname, keyname_id = keywords_to_dict()

            # get the users tweets
            tweets = user.get_tweets('tweets',10)

            followers, following = try_get_user_following(user)
            
            
            followers_list = []
            following_list = []
            found_in_user_description = False
            # add the user or their tweets to keyword_structure if a keyword is found
            for i in range(len(keyname)):
                keyword = keyname[i]
                keyword_id = keyname_id[i]
                if keyword in user.name:
                    keywords_dict[keyword_id]["ids"].append('user_'+ str(user.id))
                    relations = realtion_in_profile(user, keyword_id, relations)
                    keywords.append(keyword_id)
                    found_in_user_description = True
                elif keyword in user.screen_name:
                    keywords_dict[keyword_id]["ids"].append('user_'+ str(user.id))
                    relations = realtion_in_profile(user, keyword_id, relations)
                    keywords.append(keyword_id)
                    found_in_user_description = True
                elif keyword in user.description:
                    keywords_dict[keyword_id]["ids"].append('user_'+ str(user.id))
                    relations = realtion_in_profile(user, keyword_id, relations)
                    keywords.append(keyword_id)
                    found_in_user_description = True
                for tweet in tweets:
                    if keyword in tweet.full_text:
                        keywords_dict[keyword_id]["ids"].append('tweet_'+ str(tweet.id))
                        relations = realtion_keyword_in_tweet(tweet, keyword_id, relations)
                        if found_in_user_description:
                            pass
                        else:
                            keywords.append(keyword_id)
                            found_in_user_description = True
            if followers:
                # go through the users tweets, description, name to find keywords
                counter = 0
                for keys, keyname in keyname.items():
                    # go through the users followers description, tweet, and names to 
                    # see if it finds any keywords. If it does it adds them to the follower_list
                    followers_list, keywords_dict = keyword_search_in_users(keyname, followers, followers_list, keywords_dict, counter)

                    # Goes thorugh the users following description, tweet, and names to 
                    # see if it finds any keywords. If it does it adds them to the following_list
                    following_list, keywords_dict= keyword_search_in_users(keyname, following, following_list, keywords_dict, counter)
                    counter += 1 

                # ensure that if not followers or following where found through the keywords
                # to add however many followers are left. 
                followers_list = add_followers_or_following(followers_list, followers)
                following_list = add_followers_or_following(following_list, following)

                relations = get_relations_followers_following(relations, followers_list, user)
                relations = get_relations_followers_following(relations,following_list, user)

            # Structure to store the data
            User_Structure = {
                "username": user.screen_name,
                "user_id": 'user_' + str(user.id), 
                "followers": followers_list,
                "followees": following_list, #['user_' + str(followees_id) for followees_id in client.get_friends_ids(user.id,user.screen_name,30)]
                "profile_pic": user.profile_image_url,
                "profile_text": user.description,
                "posts": ['tweet_' + tweet.id for tweet in tweets]  ,
                "keywords": keywords
            }
            
            # go through each tweet in users tweet 
            for tweet in tweets:
                tweet_id_name = 'tweet_' + str(tweet.id)

                # check to see if tweet id not in post_strucutre if not create one for the tweet
                if tweet_id_name not in posts:

                    # extract the post structure form the tweet and comments 
                    original_pic_id_num = pic_id_num_starter
                    post_structure, picture_structure, pic_id_num_starter, relations = extract_tweet_picture_structure(user, tweet, pic_id_num_starter, relations)
                    posts[tweet_id_name] = post_structure
                    if picture_structure:
                        pictures['pic_' + str(original_pic_id_num)] = picture_structure

                replies_in_post = post_structure['comments']
                replies_in_post = [reply.replace('tweet_','') for reply in replies_in_post]
                if replies_in_post:

                    for reply in replies_in_post:
                        reply = client.get_tweet_by_id(reply)
                        reply_id_name = 'tweet_' +str(reply.id)

                        if reply_id_name not in posts:
                            original_pic_id_num = pic_id_num_starter
                            post_structure, picture_structure, pic_id_num_starter, relations = extract_tweet_picture_structure(reply.user, reply, pic_id_num_starter, relations)
                            posts[reply_id_name] = post_structure
                            
                            if picture_structure:
                                pictures['pic_' + str(original_pic_id_num)] = picture_structure

                
            # save the user to the dictionary
            user_id_name = 'user_'+ user.id
            users[user_id_name] = User_Structure
            write_txt_file(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\picture_id_num.txt', str(pic_id_num_starter))

            # list that contains all data structures 
            # data_structure_for_twitter = [users, posts, pictures, keywords, relations]
            save_json(users, save_path_list[0])
            save_json(posts, save_path_list[1])
            save_json(pictures, save_path_list[2])
            save_json(keywords_dict, save_path_list[3])
            save_json(relations, save_path_list[4])
            print(f'success with {account_num}')

            account_num += 1 
            # print('30 sec')
            # time.sleep(20)
            # print('continue')
            counter_client += 1
        except TooManyRequests as e:
            print('Rate limit exceeced')
            time_to_wait = 60 * time_counter
            time_counter += 1
            print(f"Time to wait {time_to_wait}")
            time.sleep(time_to_wait)
            if time_counter == 5:
                print('exceeded time rate limits')
                break
            counter_client += 1
            continue 


    return pic_id_num_starter

In [ ]:
def save_data_structures_to_path(save_path_list):
    keyname, keyname_id = keywords_to_dict()


    # all data structures 
    users = {}
    posts = {}
    pictures = {}
    keywords_dict = {}
    relations = []

    for keys, keynames in keyname.items():
        keyname_structure = {
            "keyword": keynames,
            "keyword_id": keyname_id[keys],
            "ids": [],
        }
        keywords_dict[keyname_id[keys]] = keyname_structure

    data_structures_to_save = [users ,posts, pictures, keywords_dict, relations]

    for i in range(len(save_path_list)):
        save_json(data_structures_to_save[i], save_path_list[i])
    return 

In [ ]:
def change_client(counter):
    if counter == 5:
        counter = 0
    if counter == 1:
        client.load_cookies('cookies4.json')
    elif counter == 2:
        client.load_cookies('cookies1.json')
    elif counter == 3:
        client.load_cookies('cookies2.json')
    elif counter == 4:
        client.load_cookies('cookies1.json')
    return 

In [ ]:
def search_user_by_keyword(keyword):
    searched_tweets = client.search_tweet(keyword)
    user_list = []
    for tweet in searched_tweets:
        user = tweet.user
        if user in user_list:
            continue
        try: 
            user_found = client.get_user_by_id(user.id)
            user_list.append(user_found)
        except:
            print('rate limits reached sleep for 2 min')
            time.sleep(120)
    return user_list

In [ ]:
save_path_list = []
keywords, keywords_ids = keywords_to_dict()
# all the file paths for all the structures in the end 
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\user_structure_2nd_round.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\post_structure__2nd_round.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\picture_strucutre__2nd_round.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\keyword_structure__2nd_round.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\relation_structure__2nd_round.json')


user_struct = open_json(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\user_structure__1st round.json')
# save_data_structures_to_path(save_path_list)
# keyname_list = [keywords[key] for key in range(13)]

counter = 1
for struct in user_struct.values():
    current_struct_work = open_json(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\user_structure_2nd_round.json')

    follower_following_list = []
    change_client(counter)
    try:
        follower_following_list.extend( [client.get_user_by_id(follower.replace('user_', '')) for count, follower in enumerate(struct['followers']) if count < 5])
        follower_following_list.extend([client.get_user_by_id(followee.replace('user_','')) for count, followee in enumerate(struct['followees']) if count < 5])
    except TooManyRequests:
        print('sleep for followers ')
        time.sleep(120)
        print('exit')
    # user_list = retry_on_rate_limit_error(client.search_user, keyname, 20)
    
    pic_id_num_starter = read_txt_file('picture_id_num.txt')

    pic_id_num_starter = creating_structures_for_twitter(follower_following_list, int(pic_id_num_starter), save_path_list)
    write_txt_file('picture_id_num.txt', str(pic_id_num_starter))
    print('')
    print('sleep')
    time.sleep(60)
    print('exit')
    counter += 1
    if len(current_struct_work) > 150:
        print('limits reached')
# pic_id_num_starter = 20000000


# [0,13] words to search. 
# list = client.search_user('Amphetamine', 20)
# list_20 = list.next()
# list_40 = list_20.next()
# list_60 = list_40.next()
# list_80 = list_60.next()

# picture_id_dict = {}



In [ ]:
save_path_list = []
keywords, keywords_ids = keywords_to_dict()
# all the file paths for all the structures in the end 
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_4th_pull\user_structure_1st_round_new_drugs.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_4th_pull\post_structure__1st_round_new_drugs.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_4th_pull\picture_strucutre__1st_round_new_drugs.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_4th_pull\keyword_structure__1st_round_new_drugs.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_4th_pull\relation_structure__1st_round_new_drugs.json')



# user_struct = open_json(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_3rd_pull\user_structure_1st_round_new_drugs.json')

# save_data_structures_to_path(save_path_list)
keyname_list = [keywords[key] for key in range(2, 26)]


counter = 1
for keyname in keyname_list:
    print(keyname)
    change_client(counter)
    tweet1 = client.search_tweet(keyname,'Latest',10)
    client_tweets = client.search_tweet(keyname,'Top', 10)
    tweet1 = [twt for twt in tweet1]
    client_tweets = [twt for twt in client_tweets]
    client_tweets.extend(tweet1)
    user_list = []
    for tweet in client_tweets:
        user = tweet.user
        user_list.append(user)
    # user_list = retry_on_rate_limit_error(client.search_user, keyname, 20)
    pic_id_num_starter = read_txt_file('picture_id_num.txt')
    pic_id_num_starter = creating_structures_for_twitter(user_list, int(pic_id_num_starter), save_path_list)
    write_txt_file('picture_id_num.txt', str(pic_id_num_starter))
    print('')
    counter += 1


In [ ]:

# counter = 0 
# for struct in user_struct.values():
#     current_struct = open_json(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_3rd_pull\user_structure_2nd_round_new_drugs.json')

#     print(keywords)
#     follower_following_list = []
#     change_client(counter)
#     try:
#         follower_following_list.extend([client.get_user_by_id(follower.replace('user_', '')) for count, follower in enumerate(struct['followers']) if count <=10])
#         follower_following_list.extend([client.get_user_by_id(followee.replace('user_','')) for count, followee in enumerate(struct['followees']) if count <=10])
#     except TooManyRequests:
#         print('sleep')
#         time.sleep(120)
#         print('exit')

    # pic_id_num_starter = read_txt_file('picture_id_num.txt')
    # tweet1 = client.search_tweet(keywords,'Latest',20)
    # client_tweets = client.search_tweet(keywords,'Top', 20)
    # tweet1 = [twt for twt in tweet1]
    # client_tweets = [twt for twt in client_tweets]
    # client_tweets.extend(tweet1)
    # user_list = []
    # for tweet in client_tweets:
    #     user = tweet.user
    #     user_list.append(user)
        

    # pic_id_num_starter = creating_structures_for_twitter(follower_following_list, int(pic_id_num_starter), save_path_list)
    # write_txt_file('picture_id_num.txt', str(pic_id_num_starter))
    # print('sleep')
    # time.sleep(60)
    # print('')
    # counter +=1
    # if len(current_struct) > 150:
    #     break


In [ ]:
user_struct_1st_round = [r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\user_structure__1st round.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_2nd_pull\user_structure_1st_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_3rd_pull\user_structure_1st_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_4th_pull\user_structure_1st_round_new_drugs.json']

post_struct_1st_round = [r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\post_structure__1st round.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_2nd_pull\post_structure__1st_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_3rd_pull\post_structure__1st_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_4th_pull\post_structure__1st_round_new_drugs.json']
                            
picture_struct_1st_round = [r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\picture_strucutre__1st round.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_2nd_pull\picture_strucutre__1st_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_3rd_pull\picture_strucutre__1st_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_4th_pull\picture_strucutre__1st_round_new_drugs.json']

relation_struct_1st_round = [r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\relation_structure__1st round.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_2nd_pull\relation_structure__1st_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_3rd_pull\relation_structure__1st_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_4th_pull\relation_structure__1st_round_new_drugs.json']

keyword_struct_1st_round = [r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\keyword_structure_1st round.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_2nd_pull\keyword_structure__1st_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_3rd_pull\keyword_structure__1st_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_4th_pull\keyword_structure__1st_round_new_drugs.json']

all_paths = [user_struct_1st_round, post_struct_1st_round, picture_struct_1st_round, relation_struct_1st_round, keyword_struct_1st_round]
for i,path_list in enumerate(all_paths):
    for path in path_list:
        if os.path.exists(path):
            pass
        else:
            print(path)

In [ ]:
user_struct_2nd_round = [r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\user_structure_2nd_round.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_2nd_pull\user_structure_2nd_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_3rd_pull\user_structure_2nd_round_new_drugs.json']

post_struct_2nd_round = [r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\post_structure__2nd_round.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_2nd_pull\post_structure__2nd_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_3rd_pull\post_structure__2nd_round_new_drugs.json']

picture_struct_2nd_round =  [r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\picture_strucutre__2nd_round.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_2nd_pull\picture_strucutre__2nd_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_3rd_pull\picture_strucutre__2nd_round_new_drugs.json']

relation_struct_2nd_round =  [r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\relation_structure__2nd_round.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_2nd_pull\relation_structure__2nd_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_3rd_pull\relation_structure__2nd_round_new_drugs.json']

keyword_struct_2nd_round = [r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\keyword_structure__2nd_round.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_2nd_pull\keyword_structure__2nd_round_new_drugs.json',
                         r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures_3rd_pull\keyword_structure__2nd_round_new_drugs.json']

all_2nd_paths = [user_struct_2nd_round, post_struct_2nd_round, picture_struct_2nd_round, relation_struct_2nd_round, keywrod_struct_2nd_round]

for path_list in all_2nd_paths:
    for path in path_list:
        if os.path.exists(path):
            pass
        else:
            print(path)

In [ ]:
def final_struct(path_list):
    struct_list = []
    for path in path_list:
        struct = open_json(path)
        struct_list.append(struct)
    
    final_struct = {}

    for struct in struct_list:
        for user in struct:
            if user in final_struct.keys():
                continue 
            final_struct[user] = struct[user]
    return final_struct

In [ ]:
def relation_final_struct(path_list):
    list_struct = []
    for path in path_list:
        struct = open_json(path)
        list_struct.append(struct)
    
    final_list = []
    for struct in list_struct:
        for relation in struct:
            if any(relation == info for info in relation):
                continue 
            final_list.append(relation)

    return final_list
            


In [ ]:
def keyword_final_struct(path_list):
    struct_list = []
    for path in path_list:
        struct = open_json(path)
        struct_list.append(struct)
    
    final_struct = {}

    # go into the first list, then go into the dictionary and then each key 
    for struct in struct_list:
        for key in struct:

            # if the key in the final structure add ids if they are not in there 
            if key in final_struct.keys():
                ids_list = final_struct[key]['ids']
                current_ids_list = struct[key]['ids']

                for cur_ids in current_ids_list:
                    if cur_ids in ids_list:
                        continue
        
                    ids_list.append(cur_ids)
                final_struct[key]['ids'] = ids_list
                
                continue 

            # if not in dictionary add it and get rid of all ids that are duplicates. 
            final_ids = set()
            for ids in struct[key]['ids']:
                if ids in final_ids:
                    continue
                final_ids.add(ids)

            struct[key]['ids'] = list(final_ids)
            final_struct[key] = struct[key]
    return final_struct
    

In [ ]:
final_data_list = [final_struct(user_struct_1st_round),
 final_struct(user_struct_2nd_round),
final_struct(post_struct_1st_round),
final_struct(post_struct_2nd_round),
final_struct(picture_struct_1st_round),
 final_struct(picture_struct_2nd_round),
 relation_final_struct(relation_struct_1st_round),
 relation_final_struct(relation_struct_2nd_round),
 keyword_final_struct(keyword_struct_1st_round),
keyword_final_struct(keyword_struct_2nd_round)]
final_struct_list = ['user_1st_round_final_struct', 'user_2nd_round_final_struct', 'post_1st_round_final_struct', 'post_2nd_round_final_struct',
                      'picture_1st_round_final_struct', 'picture_2nd_round_final_struct', 'relation_1st_round_final_struct', 'relation_2nd_round_final_struct', 
                     'keyword_1st_round_final_struct', 'keyword_2nd_round_final_struct']

for i, struct in enumerate(final_data_list):
    path = r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\final_data_structures'
    path_file = final_struct_list[i] + '.json'
    final_path = os.path.join(path,path_file)
    save_json(struct, final_path)
 


